In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This idea is copied from the following notebook which has both great implemnetation and a much better LB score:
https://www.kaggle.com/hiro5299834/tps-apr-2021-voting-pseudo-labeling

# Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_selection import RFECV 

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow import keras
from tensorflow.keras import layers


import optuna

import graphviz
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

In [ ]:
# Age fillna with mean age for each class
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
test_df['Age'] = test_df['Age'].fillna(train_df['Age'].mean())

# Cabin, fillna with 'N' and take first letter
train_df['Cabin'] = train_df['Cabin'].fillna('N').map(lambda x: x[0].strip())
test_df['Cabin'] = test_df['Cabin'].fillna('N').map(lambda x: x[0].strip())

# Ticket, fillna with 'X', split string and take first split 
train_df['Ticket'] = train_df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')
test_df['Ticket'] = test_df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

# Fare, fillna with mean value
fare_map = train_df[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
train_df['Fare'] = train_df['Fare'].fillna(train_df['Pclass'].map(fare_map['Fare']))
test_df['Fare'] = test_df['Fare'].fillna(test_df['Pclass'].map(fare_map['Fare']))

# Embarked, fillna with 'N' value
train_df['Embarked'] = train_df['Embarked'].fillna('N')
test_df['Embarked'] = test_df['Embarked'].fillna('N')

# Creating family Size Variable
train_df['family_size'] = train_df['SibSp'] + train_df['Parch']
test_df['family_size'] = test_df['SibSp'] + test_df['Parch']

In [ ]:
label_col = ['Pclass', 'Sex', 'Cabin', 'Embarked','Ticket']

lc = LabelEncoder()
for col in label_col:
    train_df[col] = lc.fit_transform(train_df[col])
    test_df[col] = lc.transform(test_df[col])

In [ ]:
features = ['Age','Fare','Pclass', 'Sex', 'Cabin', 'Embarked','Ticket','SibSp','Parch']

In [ ]:
sc = StandardScaler()
train_df[features] = sc.fit_transform(train_df[features])
test_df[features] = sc.transform(test_df[features])

# Logistic Regression Model

In [ ]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
y_pred = np.zeros(test_df.shape[0])
y_oof = np.zeros(train_df.shape[0])

train = train_df[features]
test = test_df[features]
target = train_df['Survived']
acc_mean = 0

for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
    print("Running Fold {}".format(fold + 1))
    X_train, X_valid = pd.DataFrame(train.iloc[train_index]), pd.DataFrame(train.iloc[valid_index])
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    y_oof[valid_index] = model.predict_proba(X_valid)[:,1] 
    accuracy = accuracy_score(y_valid, np.where(y_oof[valid_index]>0.5, 1, 0)) 
    print("  Accuracy: {}".format(accuracy))
    acc_mean += accuracy
    y_pred += model.predict_proba(test)[:,1]    

y_pred /= n_folds
acc_mean /= n_folds

print("")
print("Mean Accuracy: {}".format(acc_mean))
print("Done!")

In [ ]:
oof_lr = y_oof
pred_lr = y_pred

# Decision Tree Model

In [ ]:
params = {'max_depth': 10, 'min_samples_split': 818, 'min_samples_leaf': 35}

In [ ]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
y_pred = np.zeros(test_df.shape[0])
y_oof = np.zeros(train_df.shape[0])

train = train_df[features]
test = test_df[features]
target = train_df['Survived']
acc_mean = 0

for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
    print("Running Fold {}".format(fold + 1))
    X_train, X_valid = pd.DataFrame(train.iloc[train_index]), pd.DataFrame(train.iloc[valid_index])
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    model = DecisionTreeClassifier(**params)
    model.fit(X_train , y_train) 
    y_oof[valid_index] = model.predict_proba(X_valid)[:,1] 
    accuracy = accuracy_score(y_valid, np.where(y_oof[valid_index]>0.5, 1, 0)) 
    print("  Accuracy: {}".format(accuracy))
    acc_mean += accuracy
    y_pred += model.predict_proba(test)[:,1]   

y_pred /= n_folds
acc_mean /= n_folds

print("")
print("Mean Accuracy: {}".format(acc_mean))
print("Done!")

In [ ]:
oof_dt = y_oof
pred_dt = y_pred

# Random Forest Model

In [ ]:
params = {'n_estimators': 400,
    'max_depth': 80,
    'min_samples_split': 41,
    'min_samples_leaf': 36}

In [ ]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
y_pred = np.zeros(test_df.shape[0])
y_oof = np.zeros(train_df.shape[0])

train = train_df[features]
test = test_df[features]
target = train_df['Survived']
acc_mean = 0

for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
    print("Running Fold {}".format(fold + 1))
    X_train, X_valid = pd.DataFrame(train.iloc[train_index]), pd.DataFrame(train.iloc[valid_index])
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    model = RandomForestClassifier(**params)
    model.fit(X_train , y_train)
    y_oof[valid_index] = model.predict_proba(X_valid)[:,1] 
    accuracy = accuracy_score(y_valid, np.where(y_oof[valid_index]>0.5, 1, 0)) 
    print("  Accuracy: {}".format(accuracy))
    acc_mean += accuracy
    y_pred += model.predict_proba(test)[:,1]   

y_pred /= n_folds
acc_mean /= n_folds

print("")
print("Mean Accuracy: {}".format(acc_mean))
print("Done!")

In [ ]:
oof_rf = y_oof
pred_rf = y_pred

# Light GBM

In [ ]:
params = {
    'metric': 'binary_logloss',
    'n_estimators': 1000,
    'objective': 'binary',
    'learning_rate': 0.01,
    'reg_lambda': 2.343120757686286,
    'reg_alpha': 9.978349316502767,
    'colsample_bytree': 0.4,
    'min_child_samples': 100,
    'max_depth': 70,
    'num_leaves': 50,
    'device_type' : 'gpu'
}

In [ ]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
y_pred = np.zeros(test_df.shape[0])
y_oof = np.zeros(train_df.shape[0])

train = train_df[features]
test = test_df[features]
target = train_df['Survived']
acc_mean = 0

for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
    print("Running Fold {}".format(fold + 1))
    X_train, X_valid = pd.DataFrame(train.iloc[train_index]), pd.DataFrame(train.iloc[valid_index])
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    model = LGBMClassifier(**params)
    model.fit(X_train , y_train , eval_set = [(X_valid , y_valid)] ,eval_metric='logloss', early_stopping_rounds = 300 , verbose = False)
    y_oof[valid_index] = model.predict_proba(X_valid)[:,1] 
    accuracy = accuracy_score(y_valid, np.where(y_oof[valid_index]>0.5, 1, 0)) 
    print("  Accuracy: {}".format(accuracy))
    acc_mean += accuracy
    y_pred += model.predict_proba(test)[:,1]   

y_pred /= n_folds
acc_mean /= n_folds

print("")
print("Mean Accuracy: {}".format(acc_mean))
print("Done!")

In [ ]:
oof_lgbm = y_oof
pred_lgbm = y_pred

# ANN Model

In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[9]),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
y_pred = np.zeros(test_df.shape[0])
y_oof = np.zeros(train_df.shape[0])

train = train_df[features]
test = test_df[features]
target = train_df['Survived']
acc_mean = 0

for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
    print("Running Fold {}".format(fold + 1))
    X_train, X_valid = pd.DataFrame(train.iloc[train_index]), pd.DataFrame(train.iloc[valid_index])
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=512, epochs=100, callbacks=[early_stopping], verbose=0,)
    y_oof[valid_index] = model.predict(X_valid).flatten()
    accuracy = accuracy_score(y_valid, np.where(y_oof[valid_index]>0.5, 1, 0)) 
    print("  Accuracy: {}".format(accuracy))
    acc_mean += accuracy
    y_pred += model.predict(test).flatten()  

y_pred /= n_folds
acc_mean /= n_folds

print("")
print("Mean Accuracy: {}".format(acc_mean))
print("Done!")

In [ ]:
oof_ann = y_oof
pred_ann = y_pred

In [ ]:
oof_df = pd.DataFrame()
oof_df['oof_lr'] = oof_lr
oof_df['oof_dt'] = oof_dt
oof_df['oof_rf'] = oof_rf
oof_df['oof_lgbm'] = oof_lgbm
oof_df['oof_ann'] = oof_ann

In [ ]:
corrMatrix_oof = oof_df.corr()
sns.heatmap(corrMatrix_oof, annot=True)
plt.show()

In [ ]:
pred_df = pd.DataFrame()
pred_df['pred_lr'] = pred_lr
pred_df['pred_dt'] = pred_dt
pred_df['pred_rf'] = pred_rf
pred_df['pred_lgbm'] = pred_lgbm
pred_df['pred_ann'] = pred_ann

In [ ]:
corrMatrix_pr = oof_df.corr()
sns.heatmap(corrMatrix_pr, annot=True)
plt.show()

In [ ]:
oof_df['Survived'] = train_df['Survived']

In [ ]:
n_folds = 10
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
y_pred = np.zeros(test_df.shape[0])
y_oof = np.zeros(train_df.shape[0])

train = oof_df.drop('Survived', axis=1)
test = pred_df
target = oof_df['Survived']
acc_mean = 0

for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
    print("Running Fold {}".format(fold + 1))
    X_train, X_valid = pd.DataFrame(train.iloc[train_index]), pd.DataFrame(train.iloc[valid_index])
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_oof[valid_index] = model.predict_proba(X_valid)[:,1] 
    accuracy = accuracy_score(y_valid, np.where(y_oof[valid_index]>0.5, 1, 0)) 
    print("  Accuracy: {}".format(accuracy))
    acc_mean += accuracy
    y_pred += model.predict_proba(test)[:,1]   

y_pred /= n_folds
acc_mean /= n_folds

print("")
print("Mean Accuracy: {}".format(acc_mean))
print("Done!")

In [ ]:
submission['Survived'] = np.where(y_pred>0.5, 1, 0)
submission.to_csv('submission.csv',index=False)